In [ ]:
# Import libraries
import os
import pandas as pd
from pathlib import Path
import configparser
import json
from tabulate import tabulate

In [ ]:
# Initialize
base_path = Path(r'U:\WP 765 Energy RIC\Private data & analysis\Alternative Approach_Private R&D\Orbis_Data\Data_2020')

report = []

In [ ]:
# Import config parameters
config = configparser.ConfigParser(
    converters={'list': lambda x: [i.strip() for i in x.split(',')]}
)
     
config.read(base_path.joinpath(r'config.ini'))

CASE = 'EU_28'

MAPPING = Path(config.get('DEFAULT','MAPPING_PATH'))
SCREENING_KEYS = config.getlist('DEFAULT','SCREENING_KEYS')

REGION = config.getlist(CASE,'ORBIS_REGION')
CASE_ROOT = base_path.joinpath(config.get(CASE,'CASE_ROOT_PATH'))
YEAR_LASTAV = config.getint(CASE,'YEAR_LASTAV')
SUBS_ID_FILE_N = config.getint(CASE,'SUBS_ID_FILE_N')
SUBS_FIN_FILE_N = config.getint(CASE,'SUBS_FIN_FILE_N')
GROUPS_FIN_FILE_N = config.getint(CASE,'GROUPS_FIN_FILE_N')
METHOD = config.get(CASE,'SUBS_METHOD')

In [ ]:
with open(base_path.joinpath(r'Keywords.json'), 'r') as file:
    keywords = json.load(file)

# Read subsidiaries keyword screening
Sub_keyword_mask_df = pd.read_csv(
    CASE_ROOT.joinpath(r'Listed companies subsidiaries - Screening.csv')
).drop(
    columns = keywords.keys()
).rename(
    columns={'BvD_id': 'Sub_BvD_id','Company_name': 'Sub_Company_name'}
)

Sub_keyword_mask_df['Sub_Turnover_masked'] = Sub_keyword_mask_df['Sub_Turnover'].mask(~Sub_keyword_mask_df['Keyword_mask'])

In [ ]:
# Read subsidiaries to group mapping
Sub_ingroup_df = pd.read_csv(CASE_ROOT.joinpath(r'Listed companies subsidiaries.csv'))

In [ ]:
# Read ORBIS input list for groups financials
Groups_fin_df = pd.read_csv(CASE_ROOT.joinpath(r'Listed companies - Financials.csv'),
                            na_values = 'n.a.'
                           )

In [ ]:
# Read Subsidiary financials - RnD expenses
#Sub_fin_df = pd.read_excel(INPUT.joinpath(r'Subsidiaries - Financials - RnD.xlsx'),
#                             sheet_name = 'Results',
#                             names = ['Rank', 'Sub_Company_name', 'Sub_BvD_id', 'Sub_Country_ISO', 'Sub_NACE_Code', 'Sub_NACE_desc', 'Year_LastAv']
#                             + ['Sub_RnD_Y_LastAv', 'Sub_Emp_number', 'Sub_OpRev_Y_LastAv', 'Sub_NetSales_Y_LastAv']
#                             + ['Sub_RnD_Y' + str(YY) for YY in range(10,20)[::-1]],
#                             na_values = 'n.a.',
#                             dtype = {
#                                 **{col: str for col in ['Sub_Company_name', 'Sub_BvD_id', 'Country_ISO', 'NACE_Code', 'NACE_desc']},
#                                 **{col: float for col in ['RnD_Y_LastAv','Emp_number', 'OpRev_Y_LastAv', 'NetSales_Y_LastAv'] 
#                                    + ['RnD_Y' + str(YY) for YY in range(10,20)]
#                                   },
#                                 **{'Year_LastAvail': pd.Int16Dtype()}
#                             } 
#                            ).drop(columns = ['Rank', 'Sub_Country_ISO', 'Sub_NACE_Code', 'Sub_NACE_desc', 'Year_LastAv'])

In [ ]:
report.append({'': 'With financials',
               '#Performers': Groups_fin_df['BvD_id'].count().sum()
              })

In [ ]:
# Read List of performers
Group_beingSub_df = pd.read_csv(CASE_ROOT.joinpath(r'Listed companies.csv'))

Groups_fin_df = Grousp_fin_df[Groups_fin_df['BvD_id'].isin(Group_beingSub_df['BvD_id'])]

In [ ]:
report.append({'': 'That are not a subsidiary',
               '#Performers': Groups_fin_df['BvD_id'].count().sum()
              })

In [ ]:
# Merging group indentification with subsidiaries keyword screening 
Sub_exp_df = pd.merge(
    Sub_keyword_mask_df, Sub_ingroup_df,
    left_on='Sub_BvD_id', right_on='Sub_BvD_id',
    how='left'
)

# Calculating group exposure
Group_exp_df = Sub_exp_df[['BvD_id', 'Company_name','Sub_Turnover','Sub_Turnover_masked']].groupby(['BvD_id','Company_name']).sum().rename(
    columns={'Sub_Turnover': 'Total_Sub_Turnover_in_group','Sub_Turnover_masked': 'Total_Sub_Turnover_masked_in_group'}
)

Group_exp_df['Group_exp'] = Group_exp_df['Total_Sub_Turnover_masked_in_group'] / Group_exp_df['Total_Sub_Turnover_in_group']

Sub_exp_df = pd.merge(
    Sub_exp_df, Group_exp_df[['Total_Sub_Turnover_masked_in_group','Total_Sub_Turnover_in_group','Group_exp']],
    left_on='BvD_id', right_on='BvD_id',    
   how='left'
)

# Calulcating subsidiary level exposure
Sub_exp_df['Sub_exp'] = Sub_exp_df['Sub_Turnover_masked'] / Sub_exp_df['Total_Sub_Turnover_in_group']

In [ ]:
# Merging subsidiary exposure with subsidiary RnD expenses
#Sub_RnD_df = pd.merge(
#    Sub_exp_df, Sub_fin_df[['Sub_BvD_id', 'Sub_Company_name'] + ['Sub_RnD_Y' + str(YY) for YY in range(10,20)[::-1]]],
#    left_on='Sub_BvD_id', right_on='Sub_BvD_id',
#    how='left'
#)

#Sub_RnD_df = Sub_RnD_df.melt(id_vars =['Sub_BvD_id','Sub_Company_name_x','Sub_Turnover_masked','Total_Sub_Turnover_in_group','Sub_exp'],
#                  value_vars =['Sub_RnD_Y' + str(YY) for YY in range(10,20)[::-1]],
#                  var_name ='RnD_label', value_name ='Sub_RnD') 

#Sub_RnD_df['Year'] = [int('20' + s[-2:]) for s in Sub_RnD_df['RnD_label']]

#Sub_RnD_df['Sub_Clean_RnD'] = Sub_RnD_df['Sub_RnD'] * Sub_RnD_df['Sub_exp']

In [ ]:
# Merging group exposure with group RnD
Group_RnD_df = pd.merge(
    Group_exp_df, Groups_fin_df[['BvD_id', 'Company_name'] + ['RnD_Y' + str(YY) for YY in range(10,20)[::-1]]],
    left_on='BvD_id', right_on='BvD_id',
    how='left'
)

Group_RnD_df = Group_RnD_df.melt(id_vars =['BvD_id','Company_name','Total_Sub_Turnover_in_group','Total_Sub_Turnover_masked_in_group','Group_exp'],
                  value_vars =['RnD_Y' + str(YY) for YY in range(10,20)[::-1]],
                  var_name ='RnD_label', value_name ='Group_RnD') 

Group_RnD_df['Year'] = [int('20' + s[-2:]) for s in Group_RnD_df['RnD_label']]

Group_RnD_df['Group_Clean_RnD'] = Group_RnD_df['Group_RnD'] * Group_RnD_df['Group_exp']

In [ ]:
report.append({'': 'With keyword matching subsidiaries',
               '#Performers': Sub_exp_df.loc[Sub_exp_df['Keyword_mask'] == True,'BvD_id'].drop_duplicates().count().sum()
              })

report.append({'': 'With Clean RnD',
               '#Performers': Group_RnD_df.loc[Group_RnD_df['Group_Clean_RnD'] > 0,'BvD_id'].drop_duplicates().count().sum()
              })

In [ ]:
report_df = pd.DataFrame(report)

report_df = report_df[['','#Performers']]

In [ ]:
# Append report
with open(CASE_ROOT.joinpath(r'Report.txt'),'a') as report:
    report.write('#4 - Subsidiaries exposure and Group level RnD\n\n')
    report.write(tabulate(report_df, tablefmt = 'simple', headers = report_df.columns, showindex = False
                         )
                )
    report.write('\n\n')
    
# Save output tables
Sub_exp_df.to_csv(CASE_ROOT.joinpath(r'Listed companies subsidiaries - Exposure.csv'),
                  index = False,
                  float_format = '%.10f',
                  na_rep = 'n.a.',
                  columns = ['BvD_id', 'Company_name', 'Total_Sub_Turnover_masked_in_group', 'Total_Sub_Turnover_in_group',
                             'Group_exp','Sub_BvD_id','Sub_Company_name','Sub_Turnover','Keyword_mask','Sub_Turnover_masked',
                             'Sub_exp'
                            ]
                 )


Sub_exp_df[Sub_exp_df['Keyword_mask'] == True].to_csv(CASE_ROOT.joinpath(r'Listed companies subsidiaries - Exposure - Short.csv'),
                                                      index = False,
                                                      float_format = '%.10f',
                                                      na_rep = 'n.a.',
                                                      columns = ['BvD_id', 'Company_name', 'Total_Sub_Turnover_masked_in_group',
                                                                 'Total_Sub_Turnover_in_group','Group_exp','Sub_BvD_id',
                                                                 'Sub_Company_name','Sub_Turnover','Keyword_mask',
                                                                 'Sub_Turnover_masked','Sub_exp'
                                                                ]
                                                     )

#Sub_RnD_df.to_csv(OUTPUT.joinpath(r'Subsidiaries - RnD estimates - Full.csv'),
#                    index = False,
#                    columns = ['Sub_BvD_id','Sub_Company_name_x','Sub_Turnover_masked','Total_Sub_Turnover_in_group',
#                               'Sub_exp','Year','Sub_RnD','Sub_Clean_RnD'
#                              ],
#                    float_format = '%.10f',
#                    na_rep = 'n.a.'
#                   )

Group_exp_df[Group_exp_df['Group_exp'] > 0].to_csv(CASE_ROOT.joinpath(r'Listed companies - Exposure.csv'),
                   float_format = '%.10f',
                   na_rep = 'n.a.'
                   )

Groups_fin_df.to_csv(CASE_ROOT.joinpath(r'Listed companies - Financials.csv'),
                   index = False,
                   float_format = '%.10f',
                   na_rep = 'n.a.'
                   )

Group_RnD_df.to_csv(CASE_ROOT.joinpath(r'Listed companies - RnD estimates.csv'),
                    index = False,
                    columns = ['BvD_id','Company_name','Sub_Turnover_masked','Total_Sub_Turnover_in_group',
                               'Group_exp','Year','Group_RnD','Group_Clean_RnD'
                              ],
                    float_format = '%.10f',
                    na_rep = 'n.a.'
                   )

Group_RnD_df[Group_RnD_df['Group_Clean_RnD'] > 0].to_csv(CASE_ROOT.joinpath(r'Listed companies - RnD estimates - Short.csv'),
                    index = False,
                    columns = ['BvD_id','Company_name','Total_Sub_Turnover_masked_in_group','Total_Sub_Turnover_in_group',
                               'Group_exp','Year','Group_RnD','Group_Clean_RnD'
                              ],
                    float_format = '%.10f',
                    na_rep = 'n.a.'
                   )